## Análise Descritiva e Preditiva dos dados relativos ao Sistema de Operações do Crédito Rural e do Proagro (Sicor)

###  ADENILSON SILVA

## Criação da Coluna "priorizacao" e Aplicação da Técnica Downsampling

###  1 - Importando bibliotecas e criando funções

In [1]:
import pandas as pd  # Manipulação de dados tabulares (DataFrame)
from sklearn.utils import resample  # Reamostragem de dados (ex: balanceamento de classes)
import numpy as np  # Operações matemáticas e funções numéricas avançadas

### 2 - Leitura dos dados

In [2]:
df = pd.read_parquet('dados/dados_tratados/dados_tratados.parquet')
df.head(5)

,data_inicio_empreendimento,data_fim_emprendimento,finalidade,atividade,modalidade,produto,variedade,cesta_safra,consorcio,ano,taxa_juro,sigla_uf,ano_criacao_empresa,categoria_empresa
0,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2020,6.00,SC,2018,Em crescimento
1,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2023,5.00,SP,2018,Em crescimento
2,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2023,5.00,SC,2018,Em crescimento
3,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2022,4.50,ES,2018,Em crescimento
4,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2021,5.75,MA,2018,Em crescimento


### 3 - Criando coluna de Priorização com base na proporção de cada característica

In [3]:
df1 = df.atividade.value_counts(normalize=True).reset_index()
df1.columns = ['atividade', 'proporcao_atividade']

df2 = df.finalidade.value_counts(normalize=True).reset_index()
df2.columns = ['finalidade', 'proporcao_finalidade']

df3 = df.categoria_empresa.value_counts(normalize=True).reset_index()
df3.columns = ['categoria_empresa', 'proporcao_categoria_empresa']

df4 = df.modalidade.value_counts(normalize=True).reset_index()
df4.columns = ['modalidade', 'proporcao_modalidade']

df5 = df.produto.value_counts(normalize=True).reset_index()
df5.columns = ['produto', 'proporcao_produto']

cross_df = df1.merge(df2, how='cross')
cross_df = cross_df.merge(df3, how='cross')
cross_df = cross_df.merge(df4, how='cross')
cross_df = cross_df.merge(df5, how='cross')

cross_df['proporcao_conjunta'] = (
    cross_df['proporcao_atividade'] *
    cross_df['proporcao_finalidade'] *
    cross_df['proporcao_categoria_empresa'] *
    cross_df['proporcao_modalidade'] *
    cross_df['proporcao_produto']
)

colunas = ['proporcao_atividade',
           'proporcao_finalidade',
           'proporcao_categoria_empresa',
           'proporcao_modalidade',
           'proporcao_produto']

cross_df = cross_df.drop(columns=colunas)
cross_df['priorizacao'] = np.where(cross_df.proporcao_conjunta < cross_df.proporcao_conjunta.quantile(.9), 'Normal', 'Alto')

colunas_entrada = ['atividade', 'finalidade', 'categoria_empresa', 'modalidade', 'produto']
df_classificado = df.merge(cross_df, on=colunas_entrada, how='left')

df_classificado.head(5)

,data_inicio_empreendimento,data_fim_emprendimento,finalidade,atividade,modalidade,produto,variedade,cesta_safra,consorcio,ano,taxa_juro,sigla_uf,ano_criacao_empresa,categoria_empresa,proporcao_conjunta,priorizacao
0,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2020,6.00,SC,2018,Em crescimento,1.042660e-09,Normal
1,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2023,5.00,SP,2018,Em crescimento,1.042660e-09,Normal
2,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2023,5.00,SC,2018,Em crescimento,1.042660e-09,Normal
3,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2022,4.50,ES,2018,Em crescimento,1.042660e-09,Normal
4,2018-08-29,NaT,custeio,pecuario(a),pesca,armacao para barco de pesca,manutencao/aquisicao,ano civil / ano de exploracao,valor invalido,2021,5.75,MA,2018,Em crescimento,1.042660e-09,Normal


In [4]:
df_classificado['priorizacao'].value_counts().to_frame('quantidade').assign(percentual=lambda x: x['quantidade'] / x['quantidade'].sum())


,quantidade,percentual
priorizacao,,
Alto,21248813,0.98841
Normal,249163,0.01159


### 4 - Aplicação da Técnica _Downsampling_

In [5]:
df_majority = df_classificado[df_classificado['priorizacao'] == 'Alto']
df_minority = df_classificado[df_classificado['priorizacao'] == 'Normal']

n_minority = len(df_minority)
n_normal = int((n_minority * 80) / 20)

df_majority_downsampled = resample(df_majority,
                                 replace=False,   
                                 n_samples=n_normal,
                                 random_state=42) 

df_downsampled = pd.concat([df_minority, df_majority_downsampled])
df_downsampled['priorizacao'].value_counts().to_frame('quantidade').assign(percentual=lambda x: x['quantidade'] / x['quantidade'].sum())

,quantidade,percentual
priorizacao,,
Alto,996652,0.8
Normal,249163,0.2


In [6]:
colunas = ['finalidade','atividade','modalidade','produto','categoria_empresa','priorizacao']

In [7]:
df_downsampled[colunas].to_parquet('dados/dados_tratados_ML/dados_tratados_ML.parquet')
df_downsampled[colunas].to_csv('dados/dados_tratados_ML/dados_tratados_ML.csv', index=False)

### Dados sobre versão

In [8]:
import sys
import pandas as pd
import numpy as np
import sklearn as skl
print("Python:", sys.version)
print("pandas:", pd.__version__)
print("numpy:", np.__version__)
print("sklearn:", skl.__version__)

Python: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
pandas: 2.2.3
numpy: 1.23.5
sklearn: 1.3.0
